d Se Carga del archivo "Tweets.csv" y se crea el dataframe 'df'

In [2]:
#se utiliza el ejemplo dado por la plataforma para subir archivos (DBFS Example)
# File location and type
file_location = "/FileStore/tables/Tweets.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
''' Descomentar la siguiente linea para ver el DataFrame: display(df) '''
#display(df)

Out[ 10 ]: 'Descomentar la siguiente linea para ver el DataFrame: display(df) '

Se crea Tabla "Tweets_BigData" para guardar el dataframe df

In [4]:
#ahora que se tiene el dataframe, se crea una tabla para realizar los siguientes pasos de la prueba
df.createOrReplaceTempView("Tweets_BigData")

Utilizando SQL se extraen las columnas "text" y "tweet_created" de la tabla "Tweets_BigData"

In [6]:
%sql 
SELECT text, tweet_created FROM Tweets_BigData  LIMIT 10

text,tweet_created
@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800
@VirginAmerica plus you've added commercials to the experience... tacky.,2015-02-24 11:15:59 -0800
@VirginAmerica I didn't today... Must mean I need to take another trip!,2015-02-24 11:15:48 -0800
"""@VirginAmerica it's really aggressive to blast obnoxious """"entertainment"""" in your guests' faces & they have little recourse""",2015-02-24 11:15:36 -0800
@VirginAmerica and it's a really big bad thing about it,2015-02-24 11:14:45 -0800
@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.,null
null,null
"@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)",2015-02-24 11:13:57 -0800
"@VirginAmerica Really missed a prime opportunity for Men Without Hats parody, there. https://t.co/mWpG7grEZP",2015-02-24 11:12:29 -0800
"@virginamerica Well, I didn't…but NOW I DO! :-D",2015-02-24 11:11:19 -0800


Manipulacion y acondicionamiento de los datos para hallar hashtags en los textos de cada tweet, se crea el dataframe df_h

In [8]:
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql import Row

tweet_hashtags = []
text_tweets=[]
#Datos rdd, con collect se obtienen listas
text_tweet = df.select("text").rdd.collect()
#se realiza la extraccion de palabras con '#' y se reemplazan algunos caracteres sobrantes
for text in text_tweet:
  textS = str(text)
  #el ultimo replace se utiliza para eliminar los hashtags vacios
  text_m = textS.replace('")',' ').replace("')"," ").replace("."," ").replace("# ", " ")
  text_s = text_m.split(" ")
  
  hashtag = list(filter(lambda w: '#' in w, text_s )) 
  if not hashtag:
    #en caso que no haya # dentro del texto del tweet se obtiene la palabra 'not'
    text_tweets.append(list(["not"]))
  else:
    text_tweets.append(hashtag)


for tweet in text_tweets:
  #para obtener listas de strings
  cadena = str(" ".join(tweet))
  #se guarda cada lista como una fila para crear un nuevo dataframe
  tweet_hashtags.append(Row(cadena))

#se crea el esquema para el nuevo dataframe que contiene los hashtags
cSchema = StructType([StructField("Hashtags", StringType(), True)])
df_h = sqlContext.createDataFrame(tweet_hashtags, cSchema)

'''Descomentar la siguiente linea para observar el Dataframe: display(df_h)'''
#display(df_h)


Out[ 12 ]: 'Descomentar la siguiente linea para observar el Dataframe: display(df_h)'

Se crea la tabla "HashT" a partir de los datos del DataFrame df_h

In [10]:
#se crea la tabla de hashtags para realizar la consulta de los tweets mas populares utilizando count
df_h.createOrReplaceTempView("HashT")


finalmente haciendo uso de las funcionalidades de SQL, se realiza el conteo de de los hashtags agrupados y se muestran los 10 mas populares

In [12]:
%sql
SELECT  Hashtags, count(Hashtags) AS num_of_Mention FROM HashT WHERE Hashtags LIKE '#%' GROUP BY Hashtags ORDER BY num_of_Mention DESC LIMIT 10

Hashtags,num_of_Mention
#DestinationDragons,67
#fail,33
#usairwaysfail,21
#customerservice,20
#UnitedAirlines,19
#AmericanAirlines,14
#badservice,14
#help,14
#jetblue,14
#disappointed,13
